# Tiny model, 2 class

## Using pytorch lightning

In [1]:
import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt

import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger

import torchmetrics

torch.cuda.is_available()

True

In [2]:
import webdataset as wds

BATCH_SIZE = 16 # approx 400 batches

dataset = wds.WebDataset("/home/fyhuang/LocalProjects/img2loc_dataset/outputs/california_tiny.tar")\
    .decode("torchrgb").to_tuple("jpg", "json")

# Transform the labels to either ncal or scal
def to_img_label(sample):
    img, target = sample
    label = 0 if target["district"].lower() in ["alameda", "contra costa", "marin", "napa", "san francisco", "san mateo", "santa clara", "solano", "sonoma", "sacramento"] else 1
    # TODO: this is where we transform the image
    return img, label

dataset = dataset.map(to_img_label)
dataset = dataset.batched(BATCH_SIZE)

# Visualize a few loaded samples
dataloader = wds.WebLoader(dataset, batch_size=None, num_workers=0)
#dataloader = dataloader.unbatched().shuffle(1000).batched(BATCH_SIZE)

for inputs, targets in dataloader:
    print(inputs.shape, targets.shape)
    break

torch.Size([16, 3, 640, 640]) torch.Size([16])


In [3]:
# Define a LightningModule for the classifier
class NcalScalClassifierMnet3(L.LightningModule):
    def __init__(self):
        super().__init__()

        mnet3 = models.mobilenet_v3_large(weights="IMAGENET1K_V2")

        self.features = mnet3.features
        self.avgpool = mnet3.avgpool
        self.classifier = nn.Sequential(
            nn.Linear(mnet3.classifier[0].in_features, 1280),
            nn.Hardswish(inplace=True),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(1280, 2),
        )

        torch.nn.init.xavier_uniform_(self.classifier[0].weight)
        torch.nn.init.xavier_uniform_(self.classifier[3].weight)

        self.accuracy = torchmetrics.classification.Accuracy(task='multiclass', num_classes=2)

    def forward(self, x):
        with torch.no_grad():
            x = self.features(x)
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        z = self.forward(x)
        self.accuracy(z, y)
        self.log('train_acc_step', self.accuracy, prog_bar=True)

        loss = nn.CrossEntropyLoss()(z, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

clas = NcalScalClassifierMnet3()
logger = TensorBoardLogger("tiny_2class_lightning")
trainer = L.Trainer(
    max_epochs=10,
    limit_train_batches=40,
    #deterministic=True,
    logger=logger,
    log_every_n_steps=1,
)

trainer.fit(model=clas, train_dataloaders=dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params
--------------------------------------------------
0 | features   | Sequential         | 3.0 M 
1 | avgpool    | AdaptiveAvgPool2d  | 0     
2 | classifier | Sequential         | 1.2 M 
3 | accuracy   | MulticlassAccuracy | 0     
--------------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.818    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 40/40 [00:11<00:00,  3.34it/s, v_num=13, train_acc_step=0.688, train_loss=0.514]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 40/40 [00:12<00:00,  3.33it/s, v_num=13, train_acc_step=0.688, train_loss=0.514]
